In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.decomposition import PCA

In [2]:
type_precision = "float32"

In [3]:
tabula_muris_path = "../datasets/tabula_muris_whole/"
all_counts_path = "brain_mouse_matrix_all_counts.csv"
all_data_path = "brain_mouse_matrix_all_data.csv"
all_scaled_path = "brain_mouse_matrix_all_scale_data.csv"

In [4]:
all_counts = pd.read_csv(tabula_muris_path + all_counts_path, sep=" ")

In [5]:
#all_data =  pd.read_csv(tabula_muris_path + all_data_path, sep=" ")

In [6]:
#all_scaled =  pd.read_csv(tabula_muris_path + all_scaled_path, sep=" ")

In [7]:
#all_counts.describe()

In [8]:
all_counts.head()

,CELL_ID,0610005C13Rik,0610007C21Rik,0610007L01Rik,0610007N19Rik,0610007P08Rik,0610007P14Rik,0610007P22Rik,0610008F07Rik,0610009B14Rik,...,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6,zsGreen-transgene,annotation
0,A1.B003290.3_38_F.1.1,0,125,16,0,0,0,0,0,0,...,0,0,0,0,0,0,0,54,0,1
1,A1.B003728.3_56_F.1.1,0,0,0,0,0,324,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,A1.MAA000560.3_10_M.1.1,0,348,0,0,0,5,0,0,0,...,0,0,0,0,195,0,0,113,0,6
3,A1.MAA000564.3_10_M.1.1,0,41,36,0,0,24,0,0,0,...,0,0,0,125,0,1,0,0,0,4
4,A1.MAA000923.3_9_M.1.1,0,53,0,0,0,0,0,0,0,...,0,0,81,0,0,0,0,0,0,1


In [9]:
#all_counts.corr()['annotation'][:]

# Preprocessing

In [10]:
def log_normalize_data(data, scale=1000000.0):
    data_row_sums = np.sum(data, axis=1).reshape(-1, 1)
    return np.log(1 + scale * data / data_row_sums)

## Data cleaning
- one hot encoding of y
- Log normalize all data
- Split test and train data (stratified by y)
- Scale data by normal distribution

In [11]:
X = all_counts.iloc[:,1:-1].to_numpy(dtype=type_precision)
y_num = all_counts.iloc[:, -1].to_numpy(dtype=type_precision)

In [12]:
labelBin = preprocessing.LabelBinarizer()
labelBin.fit(y_num)
y = labelBin.transform(y_num)

In [13]:
X = log_normalize_data(X)

In [14]:
X_train_val, X_test, y_train_val, y_test = model_selection.train_test_split(X, y, test_size=0.33, stratify=y)

In [15]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_val)
X_train_val = scaler.transform(X_train_val)
X_test = scaler.transform(X_test)

In [16]:
print(X_train_val.shape)

(2278, 23341)


In [17]:
print(X_test.shape)

(1123, 23341)


## Dimension reduction #1 - PCA
- TODO: pick different number of components

In [18]:
num_components = 12

In [19]:
pca = PCA(n_components=num_components)

In [20]:
pca.fit(X_train_val)
X_train_val_prepared_PCA = pca.transform(X_train_val)
X_test_prepared_PCA = pca.transform(X_test)

In [21]:
print("Explained variance: {}".format(np.sum(pca.explained_variance_ratio_)))

Explained variance: 0.14409492909908295


In [22]:
X_train_val_prepared_PCA.shape, X_test_prepared_PCA.shape

((2278, 12), (1123, 12))

## Dimension reduction #2 - Autoencoder
- Problem: Autoencoder loss not decreasing at all
- Solution: increase number of layers and neurons per layer, train it on google colab

In [23]:
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [24]:
base_feature_num = X_train_val.shape[1]
encoding_dim = 12

In [25]:
base_feature_num

23341

In [26]:
X_train_val_normalized = (X_train_val-np.min(X_train_val))/(np.max(X_train_val)-np.min(X_train_val))

In [27]:
X_train_val_normalized.shape

(2278, 23341)

In [28]:
input_dim = Input(shape=(base_feature_num,))

# Encoder Layers
encoded1 = Dense(1000, activation = 'relu')(input_dim)
encoded2 = Dense(300, activation = 'relu')(encoded1)
encoded3 = Dense(encoding_dim, activation = 'relu')(encoded2)

# Decoder Layers    
decoded1 = Dense(300, activation = 'relu')(encoded3)
decoded2 = Dense(1000, activation = 'relu')(decoded1)
decoded3 = Dense(base_feature_num, activation = 'sigmoid')(decoded2)

In [29]:
autoencoder = Model(inputs=input_dim, outputs=decoded3)

In [30]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [31]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 23341)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              23342000  
_________________________________________________________________
dense_2 (Dense)              (None, 300)               300300    
_________________________________________________________________
dense_3 (Dense)              (None, 12)                3612      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               3900      
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              301000    
_________________________________________________________________
dense_6 (Dense)              (None, 23341)             23364341  
Total para

In [32]:
# autoencoder.fit(X_train_val_normalized,
#                 X_train_val_normalized,
#                 nb_epoch=1,
#                 batch_size=32,
#                 validation_split=0.2)

# Models 
- models: gradient boosting, NN, kNN(k=30 za pocetak), SVM, random forest 
- clean data
- pick dimension reduction method
- pick model
- split train and validation set

## utility functions

In [33]:
def train_model(model, data):
    X_train, y_train = data
    model.fit(X_train, y_train)
    return model

In [34]:
def evaluate_model(model, data, name):
    X_train, X_val, y_train, y_val = data
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    print('Model: {}'.format(name))
    print('Train set report: \n{}'.format(metrics.classification_report(y_train, y_train_pred)))
    print('Validation set report: \n{}'.format(metrics.classification_report(y_val, y_val_pred)))

In [35]:
def evaluate_nn(model, data, name):
    X_train, X_val, y_train, y_val = data
    y_train_pred = np.argmax(model.predict(X_train), axis=1) + 1
    y_val_pred = np.argmax(model.predict(X_val), axis=1) + 1
    print('Model: {}'.format(name))
    print('Train set report: \n{}'.format(metrics.classification_report(y_train, y_train_pred)))
    print('Validation set report: \n{}'.format(metrics.classification_report(y_val, y_val_pred)))

## import libraries

In [36]:
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import losses, optimizers

In [38]:
from sklearn import metrics

## get data

In [39]:
#X_train_val, y_train_val

In [40]:
X_train, X_val, y_train_bin, y_val_bin = model_selection.train_test_split(X_train_val_prepared_PCA, y_train_val, test_size=0.33, stratify=y_train_val)

In [41]:
y_train_num = labelBin.inverse_transform(y_train_bin)
y_val_num = labelBin.inverse_transform(y_val_bin)

In [42]:
print("X train/val shape: ", X_train.shape, X_val.shape)
print("y encoded train/val shape: ", y_train_bin.shape, y_val_bin.shape)
print("y train/val shape: ", y_train_num.shape, y_val_num.shape)

X train/val shape:  (1526, 12) (752, 12)
y encoded train/val shape:  (1526, 7) (752, 7)
y train/val shape:  (1526,) (752,)


## Multionomial logistic regression

In [43]:
from sklearn.linear_model import LogisticRegression

In [96]:
lr_clf = LogisticRegression(C = 10000)

In [97]:
lr_clf.fit(X_train, y_train_num)

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [98]:
evaluate_model(lr_clf, (X_train, X_val, y_train_num, y_val_num), "Logistic regression")

Model: Logistic regression
Train set report: 
             precision    recall  f1-score   support

        1.0       0.92      0.98      0.95       194
        2.0       0.25      0.06      0.09        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      1.00      1.00       321
        5.0       1.00      1.00      1.00       126
        6.0       1.00      1.00      1.00       706
        7.0       1.00      0.99      0.99        91

avg / total       0.98      0.98      0.98      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.92      0.98      0.95        95
        2.0       0.50      0.11      0.18         9
        3.0       0.97      1.00      0.99        35
        4.0       1.00      0.99      0.99       158
        5.0       0.98      0.98      0.98        62
        6.0       0.99      0.99      0.99       348
        7.0       0.96      1.00      0.98        45

avg / total       0.98  

## Gradient boosting

In [47]:
grad_boost_clf = ensemble.GradientBoostingClassifier(n_estimators=600, max_depth=2, learning_rate=0.006)

In [48]:
grad_boost_clf = train_model(grad_boost_clf, (X_train, y_train_num))

In [49]:
evaluate_model(grad_boost_clf, (X_train, X_val, y_train_num, y_val_num), "Grad Boost")

Model: Grad Boost
Train set report: 
             precision    recall  f1-score   support

        1.0       0.96      0.99      0.97       194
        2.0       1.00      0.44      0.62        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      1.00      1.00       321
        5.0       0.98      1.00      0.99       126
        6.0       0.99      1.00      1.00       706
        7.0       1.00      1.00      1.00        91

avg / total       0.99      0.99      0.99      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.93      0.97      0.95        95
        2.0       1.00      0.11      0.20         9
        3.0       0.97      1.00      0.99        35
        4.0       0.99      1.00      1.00       158
        5.0       0.98      1.00      0.99        62
        6.0       0.99      1.00      1.00       348
        7.0       1.00      1.00      1.00        45

avg / total       0.98      0.98 

## ADA Boosting

In [50]:
# Create and fit an AdaBoosted decision tree
ada_boost_clf = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=2),
                                            algorithm="SAMME",
                                            n_estimators=400,
                                            learning_rate=0.5)


In [51]:
ada_boost_clf = train_model(ada_boost_clf, (X_train, y_train_num))

In [52]:
evaluate_model(ada_boost_clf, (X_train, X_val, y_train_num, y_val_num), "Ada Boost")

Model: Ada Boost
Train set report: 
             precision    recall  f1-score   support

        1.0       0.95      1.00      0.97       194
        2.0       1.00      0.39      0.56        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      1.00      1.00       321
        5.0       1.00      1.00      1.00       126
        6.0       1.00      1.00      1.00       706
        7.0       1.00      1.00      1.00        91

avg / total       0.99      0.99      0.99      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.92      0.96      0.94        95
        2.0       0.00      0.00      0.00         9
        3.0       0.97      1.00      0.99        35
        4.0       0.99      1.00      1.00       158
        5.0       0.98      1.00      0.99        62
        6.0       0.99      0.99      0.99       348
        7.0       0.98      0.98      0.98        45

avg / total       0.97      0.98  

## XGBoost

In [53]:
## - pip install xgboost
## -- import xgboost as xgb

In [54]:
import xgboost as xgb

In [55]:
xgb_clf = xgb.XGBClassifier(objective='multi:softprob', max_depth=3, n_estimators=650, learning_rate=0.045)

In [56]:
xgb_clf = train_model(xgb_clf, (X_train, y_train_num))

In [57]:
evaluate_model(xgb_clf, (X_train, X_val, y_train_num, y_val_num), "kNN")

Model: kNN
Train set report: 
             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00       194
        2.0       1.00      1.00      1.00        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      1.00      1.00       321
        5.0       1.00      1.00      1.00       126
        6.0       1.00      1.00      1.00       706
        7.0       1.00      1.00      1.00        91

avg / total       1.00      1.00      1.00      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.95      0.96      0.95        95
        2.0       0.60      0.33      0.43         9
        3.0       0.97      1.00      0.99        35
        4.0       1.00      1.00      1.00       158
        5.0       1.00      1.00      1.00        62
        6.0       0.99      0.99      0.99       348
        7.0       0.98      1.00      0.99        45

avg / total       0.98      0.98      0.

/home/rastko/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rastko/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Neural network

In [58]:
number_of_features = X_train.shape[-1]
output_size = y_train_bin.shape[-1]

In [59]:
number_of_features

12

In [60]:

nn_clf = Sequential()
nn_clf.add(Dense(units=30, input_dim=number_of_features, activation='relu'))
nn_clf.add(Dense(units=100, activation='relu'))
nn_clf.add(Dense(units=30, activation='relu'))
nn_clf.add(Dense(units=output_size, activation='sigmoid'))

In [61]:
nn_clf.compile(optimizer='adam', loss=losses.binary_crossentropy, metrics=['accuracy'])

In [62]:
nn_clf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 30)                390       
_________________________________________________________________
dense_8 (Dense)              (None, 100)               3100      
_________________________________________________________________
dense_9 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 217       
Total params: 6,737
Trainable params: 6,737
Non-trainable params: 0
_________________________________________________________________


In [63]:
history = nn_clf.fit(X_train, y_train_bin, epochs=40, batch_size=30, verbose=2, validation_data=(X_val, y_val_bin))

Train on 1526 samples, validate on 752 samples
Epoch 1/40
 - 9s - loss: 0.5700 - acc: 0.7719 - val_loss: 0.1704 - val_acc: 0.9440
Epoch 2/40
 - 0s - loss: 0.0866 - acc: 0.9825 - val_loss: 0.0342 - val_acc: 0.9934
Epoch 3/40
 - 0s - loss: 0.0361 - acc: 0.9906 - val_loss: 0.0237 - val_acc: 0.9930
Epoch 4/40
 - 0s - loss: 0.0251 - acc: 0.9928 - val_loss: 0.0201 - val_acc: 0.9941
Epoch 5/40
 - 0s - loss: 0.0207 - acc: 0.9936 - val_loss: 0.0175 - val_acc: 0.9947
Epoch 6/40
 - 0s - loss: 0.0182 - acc: 0.9946 - val_loss: 0.0164 - val_acc: 0.9945
Epoch 7/40
 - 0s - loss: 0.0167 - acc: 0.9948 - val_loss: 0.0151 - val_acc: 0.9951
Epoch 8/40
 - 0s - loss: 0.0153 - acc: 0.9954 - val_loss: 0.0143 - val_acc: 0.9956
Epoch 9/40
 - 0s - loss: 0.0142 - acc: 0.9959 - val_loss: 0.0137 - val_acc: 0.9954
Epoch 10/40
 - 0s - loss: 0.0145 - acc: 0.9957 - val_loss: 0.0135 - val_acc: 0.9951
Epoch 11/40
 - 0s - loss: 0.0128 - acc: 0.9967 - val_loss: 0.0140 - val_acc: 0.9949
Epoch 12/40
 - 0s - loss: 0.0122 - acc

In [64]:
evaluate_nn(nn_clf, (X_train, X_val, y_train_num, y_val_num), "NN")

Model: NN
Train set report: 
             precision    recall  f1-score   support

        1.0       0.97      1.00      0.99       194
        2.0       1.00      0.72      0.84        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      1.00      1.00       321
        5.0       1.00      1.00      1.00       126
        6.0       1.00      1.00      1.00       706
        7.0       1.00      1.00      1.00        91

avg / total       1.00      1.00      1.00      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.96      0.98      0.97        95
        2.0       0.83      0.56      0.67         9
        3.0       0.97      1.00      0.99        35
        4.0       0.99      0.99      0.99       158
        5.0       0.98      0.98      0.98        62
        6.0       1.00      0.99      0.99       348
        7.0       0.98      1.00      0.99        45

avg / total       0.98      0.99      0.9

In [65]:
#evaluate_model(nn_clf, (X_train, X_val, y_train_bin, y_val_bin), 'NN')

## kNN

In [66]:
kNN_clf = KNeighborsClassifier(n_neighbors=6)

In [67]:
kNN_clf = train_model(kNN_clf, (X_train, y_train_num))

In [68]:
evaluate_model(kNN_clf, (X_train, X_val, y_train_num, y_val_num), "kNN")

Model: kNN
Train set report: 
             precision    recall  f1-score   support

        1.0       0.94      0.98      0.96       194
        2.0       0.89      0.44      0.59        18
        3.0       0.99      1.00      0.99        70
        4.0       1.00      1.00      1.00       321
        5.0       0.98      1.00      0.99       126
        6.0       0.99      0.99      0.99       706
        7.0       1.00      0.98      0.99        91

avg / total       0.99      0.99      0.99      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.93      0.97      0.95        95
        2.0       0.50      0.33      0.40         9
        3.0       0.95      1.00      0.97        35
        4.0       1.00      0.99      0.99       158
        5.0       1.00      1.00      1.00        62
        6.0       1.00      1.00      1.00       348
        7.0       1.00      1.00      1.00        45

avg / total       0.98      0.98      0.

## SVM

In [69]:
svm_clf = svm.SVC(C=0.004, kernel='poly', degree=2)

In [70]:
svm_clf = train_model(svm_clf, (X_train, y_train_num))

In [71]:
evaluate_model(svm_clf, (X_train, X_val, y_train_num, y_val_num), "SVM")

Model: SVM
Train set report: 
             precision    recall  f1-score   support

        1.0       0.97      0.99      0.98       194
        2.0       0.93      0.78      0.85        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      1.00      1.00       321
        5.0       1.00      1.00      1.00       126
        6.0       1.00      1.00      1.00       706
        7.0       1.00      1.00      1.00        91

avg / total       1.00      1.00      1.00      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.95      1.00      0.97        95
        2.0       0.75      0.67      0.71         9
        3.0       0.97      0.97      0.97        35
        4.0       0.99      0.99      0.99       158
        5.0       1.00      0.97      0.98        62
        6.0       1.00      0.99      1.00       348
        7.0       1.00      1.00      1.00        45

avg / total       0.99      0.99      0.

## Random forest

In [72]:
random_forest_clf = ensemble.RandomForestClassifier(max_depth=4, n_estimators=1200, criterion='gini')

In [73]:
random_forest_clf = train_model(random_forest_clf, (X_train, y_train_num))

In [74]:
evaluate_model(random_forest_clf, (X_train, X_val, y_train_num, y_val_num), "Random Forest")

Model: Random Forest
Train set report: 
             precision    recall  f1-score   support

        1.0       0.91      0.97      0.94       194
        2.0       0.00      0.00      0.00        18
        3.0       1.00      1.00      1.00        70
        4.0       1.00      0.99      0.99       321
        5.0       0.98      0.99      0.99       126
        6.0       0.98      1.00      0.99       706
        7.0       1.00      1.00      1.00        91

avg / total       0.97      0.98      0.97      1526

Validation set report: 
             precision    recall  f1-score   support

        1.0       0.93      0.97      0.95        95
        2.0       0.00      0.00      0.00         9
        3.0       1.00      1.00      1.00        35
        4.0       1.00      0.99      0.99       158
        5.0       1.00      1.00      1.00        62
        6.0       0.98      1.00      0.99       348
        7.0       0.98      1.00      0.99        45

avg / total       0.97      0.

/home/rastko/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
